In [1]:
import numpy as np

In [2]:
DIRECTIONS         = np.array([(0, -1), (1, 0), (0, 1), (-1, 0)])   # UP, RIGHT, DOWN, LEFT
BOMB_COOLDOWN_TIME = 7
TOTAL_COINS        = 9   # Depends on scenario
COLS = ROWS        = 17
BLAST              = np.array([-3, -2, -1, 1, 2, 3])

In [3]:
# Calculate BOMB_MASK one time
BOMB_MASK = np.full((COLS * ROWS, COLS, ROWS), False)

x_inside = lambda x: x > 0 and x < COLS-1
y_inside = lambda y: y > 0 and y < ROWS-1

for x in range(1, COLS-1):
        for y in range(1, ROWS-1):
            if (x % 2 == 1 or y % 2 == 1):
                explosion_spots = [(x, y)]
                if x % 2 == 1:
                    explosion_spots += [(x, y + b) for b in BLAST  if y_inside(y + b)]
                if y % 2 == 1:
                    explosion_spots += [(x + b, y) for b in BLAST  if x_inside(x + b)]
                
                explosion_spots = np.array(explosion_spots).T
                index           = x * COLS + y
                BOMB_MASK[index][explosion_spots[0], explosion_spots[1]] \
                                = True

In [42]:
(BOMB_MASK*1)[17*7 + 7]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [4]:
number_of_crates       = lambda field: np.sum(field == 1)
number_of_hidden_coins = lambda remaining_coins: TOTAL_COINS - remaining_coins

In [5]:
def time_to_reach (own_position, game_field):
    travel_times = np.full_like(game_field, 1000, dtype = int)
    travel_times[own_position] = 0
    
    # Breadth first search for travel times to all reachable spots
    frontier = [own_position]
    while len(frontier) > 0:
        current = frontier.pop(0)
        for dir in DIRECTIONS:
            neighbor = tuple(dir + np.array(current)) 
            if game_field[neighbor] == 0:   # If neighbor is a free field
                time = travel_times[current] + 1
                if travel_times[neighbor] > time:
                    travel_times[neighbor] = time
                    frontier.append(neighbor)
        
    return travel_times

In [6]:
def crate_destruction_map (game_field):
    crate_mask = game_field == 1

    return np.sum(np.logical_and(crate_mask, BOMB_MASK), axis = (1, 2)).reshape(crate_mask.shape)

In [28]:
def find_crate_bombing_spots (current_position, game_field, coins_collected):
    total_time_map            = time_to_reach(current_position, game_field) + BOMB_COOLDOWN_TIME
    hidden_coin_density       = number_of_hidden_coins(coins_collected) / number_of_crates(game_field)
    crates_destroyed          = crate_destruction_map(game_field)
    expected_coins_map        = crates_destroyed * hidden_coin_density
    coin_collection_speed_map = expected_coins_map / total_time_map
    
    print(total_time_map)
    print(hidden_coin_density)
    print(crates_destroyed)
    print(expected_coins_map)
    print(coin_collection_speed_map)
    best_spots_mask = np.isclose(coin_collection_speed_map, np.max(coin_collection_speed_map))
    best_spots      = np.array(np.where(best_spots_mask)).T
    return best_spots

### Testing the code

In [37]:
coins_discovered = 1
current_position = (15, 1)

current_field = \
np.array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1],
       [-1,  0,  0,  1,  1,  1,  0,  1,  1,  1,  1,  0,  1,  0,  0,  0,
        -1],
       [-1,  0, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  0, -1,  0,
        -1],
       [-1,  1,  0,  1,  0,  1,  1,  0,  0,  1,  1,  1,  0,  1,  1,  1,
        -1],
       [-1,  1, -1,  1, -1,  1, -1,  1, -1,  0, -1,  0, -1,  1, -1,  1,
        -1],
       [-1,  1,  0,  1,  0,  0,  0,  1,  1,  1,  0,  0,  1,  0,  1,  1,
        -1],
       [-1,  0, -1,  0, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1,
        -1],
       [-1,  0,  0,  1,  1,  1,  1,  1,  0,  1,  1,  1,  0,  1,  1,  1,
        -1],
       [-1,  1, -1,  1, -1,  1, -1,  0, -1,  0, -1,  0, -1,  1, -1,  0,
        -1],
       [-1,  1,  0,  1,  1,  1,  0,  1,  1,  1,  1,  1,  0,  0,  0,  0,
        -1],
       [-1,  1, -1,  0, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1,
        -1],
       [-1,  0,  0,  1,  1,  1,  1,  0,  1,  1,  0,  1,  1,  1,  0,  1,
        -1],
       [-1,  1, -1,  1, -1,  0, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1,
        -1],
       [-1,  1,  1,  0,  1,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        -1],
       [-1,  0, -1,  1, -1,  0, -1,  1, -1,  1, -1,  1, -1,  1, -1,  0,
        -1],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  0,  0,
        -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1]])




In [38]:
find_crate_bombing_spots (current_position, current_field, coins_discovered)

[[1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007
  1007 1007 1007]
 [1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007
  1007 1007 1007]
 [1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007
  1007 1007 1007]
 [1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007
  1007 1007 1007]
 [1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007
  1007 1007 1007]
 [1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007
  1007 1007 1007]
 [1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007
  1007 1007 1007]
 [1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007
  1007 1007 1007]
 [1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007
  1007 1007 1007]
 [1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007
  1007 1007 1007]
 [1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007 1007
  1007 1007 1007]
 [1007 100

array([[13,  5]])

In [26]:
print(np.diag(np.arange(3)))
np.argmax(np.diag(np.arange(3)))

[[0 0 0]
 [0 1 0]
 [0 0 2]]


8